In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [ ]:
df = pd.read_csv('fmnist_small.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,9,0,0,0,0,0,0,0,0,0,...,0,7,0,50,205,196,213,165,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,142,142,142,21,0,3,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,0,0,0,0,0,0,0,0,...,213,203,174,151,188,10,0,0,0,0


In [ ]:
X=df.drop('label',axis=1)
y=df.iloc[:,0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.transform(y_test)

In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y=y
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
test_dataset=CustomDataset(X_test,y_test)
train_dataset=CustomDataset(X_train,y_train)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
epochs=100
learning_rate=0.01

In [ ]:
from torch import nn
class NeuralNetwork(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.network=nn.Sequential(
        nn.Linear(num_features, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )
    def forward(self,x):
        return self.network(x)

In [ ]:
model=NeuralNetwork(X_train.shape[1])

# loss function
criterion = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr= learning_rate)

In [ ]:
len(train_loader)

150

In [ ]:
for epoch in range(epochs):
    for data, targets in train_loader:
        # forward
        output=model(data)
        loss=criterion(output,targets)
        # backward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent or adam step
        optimizer.step()
    print(f'Epochs:{epoch+1}Loss:{loss.item()}')

Epochs:1Loss:1.6716126203536987
Epochs:2Loss:0.9716033339500427
Epochs:3Loss:0.5983811020851135
Epochs:4Loss:0.5991803407669067
Epochs:5Loss:0.6346564292907715
Epochs:6Loss:0.4980531334877014
Epochs:7Loss:0.38541245460510254
Epochs:8Loss:0.5988313555717468
Epochs:9Loss:0.265322744846344
Epochs:10Loss:0.6617140769958496
Epochs:11Loss:0.4731529951095581
Epochs:12Loss:0.43459802865982056
Epochs:13Loss:0.5598965883255005
Epochs:14Loss:0.5006922483444214
Epochs:15Loss:0.23024900257587433
Epochs:16Loss:0.38498544692993164
Epochs:17Loss:0.3487391173839569
Epochs:18Loss:0.6684636473655701
Epochs:19Loss:0.1890467256307602
Epochs:20Loss:0.39589208364486694
Epochs:21Loss:0.5121266841888428
Epochs:22Loss:0.1826367825269699
Epochs:23Loss:0.3300081789493561
Epochs:24Loss:0.22884505987167358
Epochs:25Loss:0.12286550551652908
Epochs:26Loss:0.1173679530620575
Epochs:27Loss:0.3530467748641968
Epochs:28Loss:0.35936492681503296
Epochs:29Loss:0.16895505785942078
Epochs:30Loss:0.2652437686920166
Epochs:31Lo

In [ ]:
# set model to eval mode
model.eval()

NeuralNetwork(
  (network): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

    accuracy = 100.0 * n_correct / n_samples
    print(f'Accuracy: {accuracy}')

Accuracy: 83.08333333333333


torch.Size([1200])

In [ ]:
from sklearn.metrics import accuracy_score